<a href="https://colab.research.google.com/github/QuratUlAin-27/QuratUlAin-27/blob/main/Fetch_Fasta_From_Clsuter_Id.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import datetime
import shutil

# Define the target folder name
target_folder = 'core fasta'

# Get today's date
today = datetime.date.today()

# Create the target folder if it doesn't exist
if not os.path.exists(target_folder):
    os.makedirs(target_folder)
    print(f"Created folder: {target_folder}")
else:
    print(f"Folder '{target_folder}' already exists.")

# List all files in the current directory
current_files = [f for f in os.listdir('.') if os.path.isfile(f)]

moved_files_count = 0

print("\nChecking files for today's upload date...")
for filename in current_files:
    # Get the modification time of the file
    # On some systems, st_mtime (modification time) might be more reliable for 'uploaded today'
    # On others, st_ctime (creation time, or last metadata change on Unix) might be closer to 'uploaded'
    # We'll use st_mtime as it's more commonly aligned with 'last changed'
    timestamp = os.path.getmtime(filename)
    file_date = datetime.date.fromtimestamp(timestamp)

    if file_date == today:
        try:
            shutil.move(filename, os.path.join(target_folder, filename))
            print(f"Moved '{filename}' to '{target_folder}'")
            moved_files_count += 1
        except Exception as e:
            print(f"Error moving '{filename}': {e}")

if moved_files_count == 0:
    print("\nNo files uploaded today were found or moved.")
else:
    print(f"\nSuccessfully moved {moved_files_count} file(s) into '{target_folder}'.")


Created folder: core fasta

Checking files for today's upload date...
Moved '6633733907687233.pav (2).xls' to 'core fasta'
Moved '6633733907687233.table CORE GENES.xls' to 'core fasta'
Moved 'Core Fasta ' to 'core fasta'
Moved 'protein.faa' to 'core fasta'

Successfully moved 4 file(s) into 'core fasta'.


### Save 'core fasta' to Google Drive

First, we need to mount Google Drive to access its files from this Colab environment. You will be prompted to authorize this access.

In [2]:
from google.colab import drive
import os
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define the source folder (the one created earlier)
source_folder = 'core fasta'

# Define the destination path in Google Drive
# I'll create a folder named 'Colab Notebooks/core_fasta_backups' in your Drive
drive_destination_base = '/content/drive/MyDrive/Colab Notebooks'
drive_destination_folder = os.path.join(drive_destination_base, 'core_fasta_backups')

# Ensure the destination base path exists in Drive
if not os.path.exists(drive_destination_base):
    os.makedirs(drive_destination_base)
    print(f"Created base folder in Drive: {drive_destination_base}")

# Check if the source folder exists before attempting to copy
if os.path.exists(source_folder):
    # If the destination folder already exists in Drive, remove it first
    # This ensures we get a fresh copy and avoid issues with existing files
    if os.path.exists(drive_destination_folder):
        print(f"Removing existing '{drive_destination_folder}' in Drive...")
        shutil.rmtree(drive_destination_folder)

    # Copy the folder to Google Drive
    try:
        shutil.copytree(source_folder, drive_destination_folder)
        print(f"Successfully copied '{source_folder}' to '{drive_destination_folder}' in Google Drive.")
    except Exception as e:
        print(f"Error copying '{source_folder}' to Drive: {e}")
else:
    print(f"Source folder '{source_folder}' not found. Cannot copy to Drive.")


Mounted at /content/drive
Successfully copied 'core fasta' to '/content/drive/MyDrive/Colab Notebooks/core_fasta_backups' in Google Drive.


The `core fasta` directory has been copied to `Colab Notebooks/core_fasta_backups` in your Google Drive. This will ensure your files are saved. Colab notebooks are automatically saved to your Google Drive as you work.

In [9]:
!pip install Biopython
import pandas as pd
from Bio import SeqIO
import os

# Define paths for input and output files
core_genes_table_path = "core fasta/6633733907687233.table CORE GENES.xls"
all_proteins_fasta_path = "core fasta/protein.faa"
output_fasta_path = "core_genes_filtered.fasta"

# 1. Load your Core Gene data table
# Check if the core genes table exists
if not os.path.exists(core_genes_table_path):
    print(f"Error: Core genes table not found at '{core_genes_table_path}'")
else:
    # Changed to read_csv with tab separator based on error message
    clusters_df = pd.read_csv(core_genes_table_path, sep='\t')

    # Get all unique cluster IDs from the 'Cluster' column (first column)
    core_protein_ids = clusters_df['Cluster'].unique().tolist()
    print(f"Found {len(core_protein_ids)} core cluster IDs from '{core_genes_table_path}'.")

    # 2. Filter the main FASTA file for these core cluster IDs
    # Check if the all proteins fasta file exists
    if not os.path.exists(all_proteins_fasta_path):
        print(f"Error: All proteins FASTA file not found at '{all_proteins_fasta_path}'")
    else:
        with open(output_fasta_path, "w") as output_handle:
            # Parse the main FASTA file and write only the sequences whose IDs match the core cluster IDs
            for record in SeqIO.parse(all_proteins_fasta_path, "fasta"):
                if record.id in core_protein_ids:
                    SeqIO.write(record, output_handle, "fasta")
        print(f"Successfully filtered core gene sequences to '{output_fasta_path}'.")
        print(f"You can find the filtered FASTA file named '{output_fasta_path}' in your current directory.")


Found 2858 core cluster IDs from 'core fasta/6633733907687233.table CORE GENES.xls'.
Successfully filtered core gene sequences to 'core_genes_filtered.fasta'.
You can find the filtered FASTA file named 'core_genes_filtered.fasta' in your current directory.


# Task
Load the gene presence/absence variation (PAV) table from 'core fasta/6633733907687233.pav (2).xls', identify 'core' genes (present in all genomes), then use these core gene identifiers to filter the 'core fasta/protein.faa' file, saving the resulting core gene sequences into 'core_genes_final.fasta' within a new 'core_gene_fasta_output' directory, and finally copy this entire output directory to Google Drive.

### Verification: Exploring a single core gene cluster

Let's pick one `ClutserID` from our identified core genes and see how many `WP_` identifiers it has in the PAV table across different genomes. This will explain why `2858` core clusters can result in `6117` individual protein sequences.

In [18]:
import numpy as np

# Get the first core gene ID from our previously identified list
# (Assuming core_gene_ids_from_pav and pav_df_cleaned are still in memory)
if 'core_gene_ids_from_pav' in locals() and core_gene_ids_from_pav:
    sample_core_cluster_id = core_gene_ids_from_pav[0]
    print(f"Examining core ClutserID: {sample_core_cluster_id}")

    # Filter the cleaned PAV DataFrame for this specific cluster ID
    single_cluster_row = pav_df_cleaned[pav_df_cleaned['ClutserID'] == sample_core_cluster_id]

    if not single_cluster_row.empty:
        # Extract all non-NaN WP_ IDs from the genome columns for this cluster
        wp_ids_for_sample_cluster = set()
        for col in genome_columns:
            # Get the value from the current genome column for this cluster
            item = single_cluster_row[col].iloc[0]
            if pd.notna(item):
                # Split by comma in case of multiple WP_ IDs in one cell
                for sub_item in str(item).split(','):
                    wp_ids_for_sample_cluster.add(sub_item.strip())

        print(f"\nWP_ IDs associated with ClutserID {sample_core_cluster_id} across genomes:")
        for i, wp_id in enumerate(list(wp_ids_for_sample_cluster)):
            if i < 10: # Print first 10 for brevity
                print(f"- {wp_id}")
            elif i == 10:
                print("... (and more)")
        print(f"Total unique WP_ IDs found for this cluster: {len(wp_ids_for_sample_cluster)}")

        # Optional: Verify if these WP_ IDs were among the ones used for filtering
        # (wp_ids_from_core_pav should contain all of these)
        # Note: 'wp_ids_from_core_pav' should be available from a previous cell's execution
        if 'wp_ids_from_core_pav' in locals():
            all_found = True
            for wp_id in wp_ids_for_sample_cluster:
                if wp_id not in wp_ids_from_core_pav:
                    all_found = False
                    break
            if all_found:
                print(f"\nAll {len(wp_ids_for_sample_cluster)} WP_ IDs for ClutserID {sample_core_cluster_id} were part of the {len(wp_ids_from_core_pav)} total unique WP_ IDs used for filtering.")
            else:
                print("\nSome WP_ IDs for this cluster were NOT found in the main filtering set. This indicates an issue.")

    else:
        print(f"ClutserID {sample_core_cluster_id} not found in pav_df_cleaned.")
else:
    print("Previous variables (core_gene_ids_from_pav, pav_df_cleaned) not found. Please run previous cells.")

Examining core ClutserID: 393

WP_ IDs associated with ClutserID 393 across genomes:
- WP_017329728.1
- WP_027791339.1
- WP_249473006.1
- WP_011355492.1
- WP_006399860.1
- WP_069259698.1
- WP_059738577.1
- WP_447361690.1
- WP_084908455.1
- WP_012338331.1
... (and more)
Total unique WP_ IDs found for this cluster: 16

All 16 WP_ IDs for ClutserID 393 were part of the 47514 total unique WP_ IDs used for filtering.


## Load and Inspect PAV Table

### Subtask:
Load the gene presence/absence variation (PAV) table from 'core fasta/6633733907687233.pav (2).xls' into a pandas DataFrame. This step will also inspect the table's structure (e.g., by displaying the head and columns) to understand how to identify core genes.


## Load and Inspect PAV Table

### Subtask:
Load the gene presence/absence variation (PAV) table from 'core fasta/6633733907687233.pav (2).xls' into a pandas DataFrame. This step will also inspect the table's structure (e.g., by displaying the head and columns) to understand how to identify core genes.

#### Instructions
1. Define the file path for the PAV table: 'core fasta/6633733907687233.pav (2).xls'.
2. Use pandas to read this Excel file into a DataFrame. You might need to specify the engine if it's an old .xls format (e.g., `engine='xlrd'`).
3. Display the first few rows of the DataFrame to understand its structure and content.
4. Print the column names of the DataFrame to identify relevant columns for core gene identification.

**Reasoning**:
Now, I will load the PAV table into a pandas DataFrame, display its head, and print its columns as instructed in the markdown cell to prepare for identifying core genes.



In [12]:
import pandas as pd
import os

# 1. Define the file path for the PAV table
pav_table_path = 'core fasta/6633733907687233.pav (2).xls'

# Check if the file exists
if not os.path.exists(pav_table_path):
    print(f"Error: PAV table not found at '{pav_table_path}'")
else:
    # 2. Use pandas to read this Excel file into a DataFrame
    # Try reading as tab-separated CSV first, as .xls extension can be misleading and 'excel' engine sometimes has issues.
    # If that fails, consider using engine='xlrd' or 'openpyxl' for actual Excel files.
    try:
        pav_df = pd.read_csv(pav_table_path, sep='\t')
        print(f"Successfully loaded '{pav_table_path}' as a tab-separated file.")
    except Exception as e:
        print(f"Could not load as tab-separated, trying with excel engine: {e}")
        try:
            # Install openpyxl if not already installed for .xlsx, or xlrd for .xls
            # !pip install openpyxl xlrd # Uncomment if needed and run in a separate cell
            pav_df = pd.read_excel(pav_table_path, engine='xlrd') # For .xls files
            print(f"Successfully loaded '{pav_table_path}' using 'xlrd' engine.")
        except ImportError:
            print("Error: 'xlrd' library not found. Please install it using '!pip install xlrd' if this is an old .xls file.")
            pav_df = None
        except Exception as e_excel:
            print(f"Error loading '{pav_table_path}' as an Excel file: {e_excel}")
            pav_df = None

    if pav_df is not None:
        # 3. Display the first few rows of the DataFrame
        print("\nFirst 5 rows of the PAV table:")
        print(pav_df.head())

        # 4. Print the column names of the DataFrame
        print("\nColumn names of the PAV table:")
        print(pav_df.columns.tolist())

Successfully loaded 'core fasta/6633733907687233.pav (2).xls' as a tab-separated file.

First 5 rows of the PAV table:
   ClutserID Burkholderia_vietnamiensis_FDAARGOS_1488_1  \
0      20005                                          -   
1      18096                                          -   
2       5808                                          -   
3        231                                          -   
4      13039                                          -   

  Burkholderia_vietnamiensis_LMG_10929_I  \
0                                      -   
1                                      -   
2                                      -   
3                                      -   
4                                      -   

  Burkholderia_vietnamiensis_FDAARGOS_239_1  \
0                                         -   
1                                         -   
2                                         -   
3                                         -   
4                         

**Reasoning**:
Now that the PAV table is loaded and its structure is understood, the next step is to identify the core genes. Core genes are defined as those present in all genomes, which means checking for the absence of the '-' character in all genome-specific columns for each gene cluster. I will filter the `pav_df` to find rows where no genome columns contain a '-', extract the 'ClutserID' from these rows, and store them as a list of core gene IDs.



In [13]:
import numpy as np

# Identify columns that represent genomes (all columns except 'ClutserID')
genome_columns = [col for col in pav_df.columns if col != 'ClutserID']

# Filter the DataFrame to find rows where all genome columns do not contain '-'
# We replace '-' with NaN to use .isnull().all() for cleaner filtering
pav_df_cleaned = pav_df.replace('-', np.nan)

# A gene is 'core' if it is present (not NaN) in ALL genome columns
core_genes_pav_df = pav_df_cleaned[pav_df_cleaned[genome_columns].notna().all(axis=1)]

# Extract the 'ClutserID' for these core genes
core_gene_ids_from_pav = core_genes_pav_df['ClutserID'].tolist()

print(f"Found {len(core_gene_ids_from_pav)} core gene IDs from the PAV table.")
print("First 5 core gene IDs:")
print(core_gene_ids_from_pav[:5])

Found 2858 core gene IDs from the PAV table.
First 5 core gene IDs:
[393, 21759, 21734, 6679, 13115]


**Reasoning**:
The core gene IDs from the PAV table (`core_gene_ids_from_pav`) need to be converted to the 'CLUSTER' format (e.g., 'CLUSTER0393') to match the protein IDs in the FASTA file, similar to how `core_protein_ids` was formatted previously. This conversion is crucial for accurate filtering of the FASTA file. After ensuring compatibility, I will proceed to filter the `all_proteins_fasta_path` FASTA file using these standardized core gene IDs and save the output to `core_genes_final.fasta` within a newly created `core_gene_fasta_output` directory, as specified in the task.



In [14]:
from Bio import SeqIO
import os

# Convert core_gene_ids_from_pav to the 'CLUSTER' format
# Assuming 'ClutserID' from the PAV table are integers that need padding and prefix.
# The previous `core_protein_ids` variable used 'CLUSTER' prefix with zero-padded numbers.
# We need to replicate that formatting.
formatted_core_gene_ids = set()
for gene_id in core_gene_ids_from_pav:
    # Pad with leading zeros to ensure a consistent length, e.g., CLUSTER0393
    # The previous `core_protein_ids` in kernel state shows 'CLUSTER0393', 'CLUSTER21759'
    # which suggests padding to 4 or 5 digits, let's go with 5 for safety, and remove excess zeros for smaller numbers.
    # However, a simpler and more robust approach is to convert the number to string and prefix 'CLUSTER'
    # as the filtering is likely based on exact string match.
    formatted_core_gene_ids.add(f"CLUSTER{gene_id:05d}") # Pad to 5 digits, e.g., 393 -> 00393

# Define paths for input and output files/directories
all_proteins_fasta_path = "core fasta/protein.faa"
output_directory = "core_gene_fasta_output"
output_fasta_filename = "core_genes_final.fasta"
output_fasta_path = os.path.join(output_directory, output_fasta_filename)

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    print(f"Created directory: {output_directory}")
else:
    print(f"Directory '{output_directory}' already exists.")

# Filter the main FASTA file for these core cluster IDs
if not os.path.exists(all_proteins_fasta_path):
    print(f"Error: All proteins FASTA file not found at '{all_proteins_fasta_path}'")
else:
    filtered_count = 0
    with open(output_fasta_path, "w") as output_handle:
        for record in SeqIO.parse(all_proteins_fasta_path, "fasta"):
            # The record.id from protein.faa is typically the Cluster ID in the expected format.
            # We need to check if the record.id (e.g., CLUSTER0393) is in our formatted set.
            if record.id in formatted_core_gene_ids:
                SeqIO.write(record, output_handle, "fasta")
                filtered_count += 1
    print(f"Successfully filtered {filtered_count} core gene sequences to '{output_fasta_path}'.")
    print(f"You can find the filtered FASTA file in the '{output_directory}' directory.")

Created directory: core_gene_fasta_output
Successfully filtered 0 core gene sequences to 'core_gene_fasta_output/core_genes_final.fasta'.
You can find the filtered FASTA file in the 'core_gene_fasta_output' directory.


**Reasoning**:
The previous filtering step failed because of a mismatch in how core gene IDs were formatted. The `core_gene_ids_from_pav` are integers (e.g., 393), but the protein IDs in the FASTA file are likely formatted as strings (e.g., 'CLUSTER0393', 'CLUSTER21759'), as seen in the `clusters_df` and `core_protein_ids` variables. To fix this, I will create a mapping from the integer IDs to their correctly formatted 'CLUSTER' string counterparts based on the `clusters_df` DataFrame. Then, I will use these correctly formatted IDs to re-filter the `protein.faa` file and save the output to `core_genes_final.fasta`. Finally, I will copy the output directory to Google Drive to complete the task.



In [15]:
from Bio import SeqIO
import os

# Create a mapping from integer ID to formatted string ID from clusters_df
id_mapping = {}
for cluster_str in clusters_df['Cluster'].unique():
    # Extract the numeric part, assuming it starts after 'CLUSTER'
    numeric_id_str = cluster_str.replace('CLUSTER', '')
    try:
        numeric_id = int(numeric_id_str)
        id_mapping[numeric_id] = cluster_str
    except ValueError:
        # This should ideally not happen if the format is consistent
        print(f"Warning: Could not parse numeric ID from '{cluster_str}'")

# Now use this mapping to format the core_gene_ids_from_pav
formatted_core_gene_ids = set()
for gene_id in core_gene_ids_from_pav:
    if gene_id in id_mapping:
        formatted_core_gene_ids.add(id_mapping[gene_id])
    else:
        # Fallback for IDs not found in the original clusters_df, though this indicates an inconsistency
        print(f"Warning: Gene ID {gene_id} from PAV table not found in original clusters_df for formatting. Using default padding.")
        # Use padding to 5 digits as a reasonable guess if specific format is unknown
        formatted_core_gene_ids.add(f"CLUSTER{gene_id:05d}")

print(f"Number of correctly formatted core gene IDs: {len(formatted_core_gene_ids)}")
print("First 5 formatted core gene IDs:")
print(list(formatted_core_gene_ids)[:5])

# Define paths for input and output files/directories
all_proteins_fasta_path = "core fasta/protein.faa"
output_directory = "core_gene_fasta_output"
output_fasta_filename = "core_genes_final.fasta"
output_fasta_path = os.path.join(output_directory, output_fasta_filename)

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    print(f"Created directory: {output_directory}")
else:
    print(f"Directory '{output_directory}' already exists.")

# Filter the main FASTA file for these core cluster IDs
if not os.path.exists(all_proteins_fasta_path):
    print(f"Error: All proteins FASTA file not found at '{all_proteins_fasta_path}'")
else:
    filtered_count = 0
    with open(output_fasta_path, "w") as output_handle:
        for record in SeqIO.parse(all_proteins_fasta_path, "fasta"):
            # Check if the record.id (e.g., CLUSTER0393) is in our correctly formatted set.
            if record.id in formatted_core_gene_ids:
                SeqIO.write(record, output_handle, "fasta")
                filtered_count += 1
    print(f"Successfully filtered {filtered_count} core gene sequences to '{output_fasta_path}'.")
    print(f"You can find the filtered FASTA file in the '{output_directory}' directory.")


# Copy the output directory to Google Drive
from google.colab import drive
import shutil

# Ensure Google Drive is mounted (if not already)
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Define the destination path in Google Drive
drive_destination_base = '/content/drive/MyDrive/Colab Notebooks'
drive_destination_folder = os.path.join(drive_destination_base, 'core_gene_fasta_output_backup') # Use a new name to avoid conflict/overwrite

# Ensure the destination base path exists in Drive
if not os.path.exists(drive_destination_base):
    os.makedirs(drive_destination_base)
    print(f"Created base folder in Drive: {drive_destination_base}")

# If the destination folder already exists in Drive, remove it first to ensure a fresh copy
if os.path.exists(drive_destination_folder):
    print(f"Removing existing '{drive_destination_folder}' in Drive...")
    shutil.rmtree(drive_destination_folder)

# Copy the folder to Google Drive
try:
    shutil.copytree(output_directory, drive_destination_folder)
    print(f"Successfully copied '{output_directory}' to '{drive_destination_folder}' in Google Drive.")
except Exception as e:
    print(f"Error copying '{output_directory}' to Drive: {e}")

Number of correctly formatted core gene IDs: 2858
First 5 formatted core gene IDs:
['CLUSTER6467', 'CLUSTER0412', 'CLUSTER18465', 'CLUSTER3540', 'CLUSTER13778']
Directory 'core_gene_fasta_output' already exists.
Successfully filtered 0 core gene sequences to 'core_gene_fasta_output/core_genes_final.fasta'.
You can find the filtered FASTA file in the 'core_gene_fasta_output' directory.
Successfully copied 'core_gene_fasta_output' to '/content/drive/MyDrive/Colab Notebooks/core_gene_fasta_output_backup' in Google Drive.


**Reasoning**:
The previous filtering attempt resulted in 0 filtered sequences, indicating a mismatch in protein ID formats. To resolve this, I will explicitly inspect the first few `record.id` entries from `protein.faa` to confirm their exact format. Based on this direct inspection, I will adjust the `formatted_core_gene_ids` to ensure they precisely match the FASTA file's ID structure, then re-attempt the filtering and the Google Drive copy.



In [16]:
from Bio import SeqIO
import os
import shutil

# 1. Verify the format of record.id in protein.faa
all_proteins_fasta_path = "core fasta/protein.faa"
if not os.path.exists(all_proteins_fasta_path):
    print(f"Error: All proteins FASTA file not found at '{all_proteins_fasta_path}'")
else:
    print("\nFirst 10 record IDs from protein.faa:")
    temp_count = 0
    protein_faa_ids_sample = []
    for record in SeqIO.parse(all_proteins_fasta_path, "fasta"):
        protein_faa_ids_sample.append(record.id)
        print(record.id)
        temp_count += 1
        if temp_count >= 10:
            break

    if not protein_faa_ids_sample:
        print("Warning: Could not read any records from protein.faa. File might be empty or corrupted.")

    # Based on the inspection, if IDs are "CLUSTERXXXX" then proceed with the existing formatting logic.
    # If they are "WP_...", then a different mapping strategy is needed.
    # Given the previous successful run of `Ae7llDxzB6ar` which used CLUSTER IDs for filtering,
    # it's highly likely they are CLUSTER IDs. The previous 'fasta_record_ids' kernel variable
    # might have been from a different context/file.

    # Re-using the id_mapping and formatted_core_gene_ids logic from the previous cell,
    # as it correctly maps integer IDs from PAV to CLUSTER strings based on clusters_df.
    # Assuming clusters_df is still in kernel and correctly populated from 'CORE GENES.xls'
    # and pav_df, core_gene_ids_from_pav are also available.

    # Create a mapping from integer ID to formatted string ID from clusters_df
    id_mapping = {}
    for cluster_str in clusters_df['Cluster'].unique():
        numeric_id_str = cluster_str.replace('CLUSTER', '')
        try:
            numeric_id = int(numeric_id_str)
            id_mapping[numeric_id] = cluster_str
        except ValueError:
            print(f"Warning: Could not parse numeric ID from '{cluster_str}' in clusters_df.")

    formatted_core_gene_ids = set()
    for gene_id in core_gene_ids_from_pav:
        if gene_id in id_mapping:
            formatted_core_gene_ids.add(id_mapping[gene_id])
        else:
            # Fallback in case a gene_id from PAV is not found in clusters_df for formatting
            # This indicates an inconsistency, but we'll try to format it anyway, similar to the original logic
            formatted_core_gene_ids.add(f"CLUSTER{gene_id:05d}")

    print(f"\nNumber of correctly formatted core gene IDs derived from PAV: {len(formatted_core_gene_ids)}")
    print("First 5 formatted core gene IDs:")
    print(list(formatted_core_gene_ids)[:5])

    # Define paths for output files/directories
    output_directory = "core_gene_fasta_output"
    output_fasta_filename = "core_genes_final.fasta"
    output_fasta_path = os.path.join(output_directory, output_fasta_filename)

    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        print(f"Created directory: {output_directory}")
    else:
        print(f"Directory '{output_directory}' already exists.")
        # If directory exists, ensure the output file is cleared before writing
        if os.path.exists(output_fasta_path):
            os.remove(output_fasta_path)
            print(f"Cleared existing '{output_fasta_path}'.")

    # Filter the main FASTA file for these core cluster IDs
    filtered_count = 0
    with open(output_fasta_path, "w") as output_handle:
        for record in SeqIO.parse(all_proteins_fasta_path, "fasta"):
            if record.id in formatted_core_gene_ids:
                SeqIO.write(record, output_handle, "fasta")
                filtered_count += 1
    print(f"Successfully filtered {filtered_count} core gene sequences to '{output_fasta_path}'.")
    print(f"You can find the filtered FASTA file in the '{output_directory}' directory.")

    # Copy the output directory to Google Drive
    # Ensure Google Drive is mounted (if not already)
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

    drive_destination_base = '/content/drive/MyDrive/Colab Notebooks'
    drive_destination_folder = os.path.join(drive_destination_base, 'core_gene_fasta_output_backup')

    if not os.path.exists(drive_destination_base):
        os.makedirs(drive_destination_base)
        print(f"Created base folder in Drive: {drive_destination_base}")

    if os.path.exists(drive_destination_folder):
        print(f"Removing existing '{drive_destination_folder}' in Drive...")
        shutil.rmtree(drive_destination_folder)

    try:
        shutil.copytree(output_directory, drive_destination_folder)
        print(f"Successfully copied '{output_directory}' to '{drive_destination_folder}' in Google Drive.")
    except Exception as e:
        print(f"Error copying '{output_directory}' to Drive: {e}")


First 10 record IDs from protein.faa:
WP_004186391.1
WP_004186661.1
WP_004186709.1
WP_004189550.1
WP_004191477.1
WP_004192636.1
WP_004192938.1
WP_004196460.1
WP_004197638.1
WP_004197937.1

Number of correctly formatted core gene IDs derived from PAV: 2858
First 5 formatted core gene IDs:
['CLUSTER6467', 'CLUSTER0412', 'CLUSTER18465', 'CLUSTER3540', 'CLUSTER13778']
Directory 'core_gene_fasta_output' already exists.
Cleared existing 'core_gene_fasta_output/core_genes_final.fasta'.
Successfully filtered 0 core gene sequences to 'core_gene_fasta_output/core_genes_final.fasta'.
You can find the filtered FASTA file in the 'core_gene_fasta_output' directory.
Removing existing '/content/drive/MyDrive/Colab Notebooks/core_gene_fasta_output_backup' in Drive...
Successfully copied 'core_gene_fasta_output' to '/content/drive/MyDrive/Colab Notebooks/core_gene_fasta_output_backup' in Google Drive.


**Reasoning**:
The previous attempt failed because the IDs extracted from the PAV table (`CLUSTER` formatted) did not match the actual protein IDs in `protein.faa` (`WP_` formatted). To correctly filter, I need to collect all `WP_` IDs from the `pav_df` that correspond to the identified core gene clusters. I will iterate through the core gene rows in the `pav_df`, extract all non-NaN (`WP_` formatted) protein IDs from the genome columns, and use this new set of `WP_` IDs to filter the `protein.faa` file. This will ensure an accurate match and successful filtering.



In [17]:
from Bio import SeqIO
import os
import shutil
import numpy as np

# Assuming pav_df, core_gene_ids_from_pav, and genome_columns are still in kernel memory

# 1. Identify all WP_ IDs associated with core gene clusters from pav_df
# Filter pav_df for only the core gene IDs identified earlier
core_pav_records = pav_df[pav_df['ClutserID'].isin(core_gene_ids_from_pav)]

# Extract all unique WP_ IDs from the genome columns of these core records
# First, replace '-' with np.nan for easier processing
core_pav_records_cleaned = core_pav_records.replace('-', np.nan)

# Collect all non-NaN values from the genome columns
# Use a set for efficient lookup during filtering
wp_ids_from_core_pav = set()
for col in genome_columns:
    # Drop NaN values and iterate through the remaining (WP_ IDs)
    for item in core_pav_records_cleaned[col].dropna():
        # Some cells might contain multiple WP_ IDs separated by commas
        for sub_item in str(item).split(','):
            wp_ids_from_core_pav.add(sub_item.strip())

print(f"Found {len(wp_ids_from_core_pav)} unique WP_ IDs associated with core gene clusters.")
print("First 10 WP_ IDs collected from core PAV records:")
print(list(wp_ids_from_core_pav)[:10])


# Define paths for input and output files/directories
all_proteins_fasta_path = "core fasta/protein.faa"
output_directory = "core_gene_fasta_output"
output_fasta_filename = "core_genes_final.fasta"
output_fasta_path = os.path.join(output_directory, output_fasta_filename)

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    print(f"Created directory: {output_directory}")
else:
    print(f"Directory '{output_directory}' already exists.")
    # If directory exists, ensure the output file is cleared before writing
    if os.path.exists(output_fasta_path):
        os.remove(output_fasta_path)
        print(f"Cleared existing '{output_fasta_path}'.")

# Filter the main FASTA file for these collected WP_ IDs
if not os.path.exists(all_proteins_fasta_path):
    print(f"Error: All proteins FASTA file not found at '{all_proteins_fasta_path}'")
else:
    filtered_count = 0
    with open(output_fasta_path, "w") as output_handle:
        for record in SeqIO.parse(all_proteins_fasta_path, "fasta"):
            if record.id in wp_ids_from_core_pav:
                SeqIO.write(record, output_handle, "fasta")
                filtered_count += 1
    print(f"Successfully filtered {filtered_count} core gene sequences to '{output_fasta_path}'.")
    print(f"You can find the filtered FASTA file in the '{output_directory}' directory.")

# Copy the output directory to Google Drive
# Ensure Google Drive is mounted (if not already)
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

drive_destination_base = '/content/drive/MyDrive/Colab Notebooks'
drive_destination_folder = os.path.join(drive_destination_base, 'core_gene_fasta_output_backup')

if not os.path.exists(drive_destination_base):
    os.makedirs(drive_destination_base)
    print(f"Created base folder in Drive: {drive_destination_base}")

# If the destination folder already exists in Drive, remove it first to ensure a fresh copy
if os.path.exists(drive_destination_folder):
    print(f"Removing existing '{drive_destination_folder}' in Drive...")
    shutil.rmtree(drive_destination_folder)

try:
    shutil.copytree(output_directory, drive_destination_folder)
    print(f"Successfully copied '{output_directory}' to '{drive_destination_folder}' in Google Drive.")
except Exception as e:
    print(f"Error copying '{output_directory}' to Drive: {e}")

Found 47514 unique WP_ IDs associated with core gene clusters.
First 10 WP_ IDs collected from core PAV records:
['WP_045577944.1', 'WP_212181163.1', 'WP_039343251.1', 'WP_069256160.1', 'WP_047898827.1', 'WP_035973219.1', 'WP_342706166.1', 'WP_059885304.1', 'WP_034190222.1', 'WP_238486775.1']
Directory 'core_gene_fasta_output' already exists.
Cleared existing 'core_gene_fasta_output/core_genes_final.fasta'.
Successfully filtered 6117 core gene sequences to 'core_gene_fasta_output/core_genes_final.fasta'.
You can find the filtered FASTA file in the 'core_gene_fasta_output' directory.
Removing existing '/content/drive/MyDrive/Colab Notebooks/core_gene_fasta_output_backup' in Drive...
Successfully copied 'core_gene_fasta_output' to '/content/drive/MyDrive/Colab Notebooks/core_gene_fasta_output_backup' in Google Drive.


## Final Task

### Subtask:
Confirm the successful completion of the entire task, including the identification of core genes, accurate filtering of FASTA sequences, and saving the results both locally and securely to Google Drive.


## Summary:

### Data Analysis Key Findings

*   The gene presence/absence variation (PAV) table from 'core fasta/6633733907687233.pav (2).xls' was successfully loaded and inspected, revealing `ClutserID` and genome-specific protein identifiers (e.g., `WP_...`) or absence markers (`-`).
*   A total of 2858 core gene identifiers were successfully extracted from the PAV table by identifying genes present in all genomes.
*   Initial attempts to filter the 'protein.faa' file using a `CLUSTER` ID format resulted in 0 sequences being filtered, indicating a mismatch between the assumed and actual FASTA header formats.
*   Upon further inspection, it was confirmed that the 'protein.faa' file utilized `WP_` formatted IDs (e.g., `WP_004186391.1`), not `CLUSTER` IDs.
*   After correcting the identifier extraction strategy, 47514 unique `WP_` IDs associated with the identified core genes were successfully collected from the PAV table.
*   Using the `WP_` IDs, 6117 core gene sequences were successfully filtered from 'protein.faa' and saved to 'core\_gene\_fasta\_output/core\_genes\_final.fasta'.
*   The resulting 'core\_gene\_fasta\_output' directory, containing 'core\_genes\_final.fasta', was successfully copied to Google Drive at '/content/drive/MyDrive/Colab Notebooks/core\_gene\_fasta\_output\_backup'.

### Insights or Next Steps

*   Always perform an initial inspection of all input data files to confirm expected formats and identifiers before proceeding with downstream analysis, especially when combining data from different sources. This can prevent significant time loss from debugging incorrect assumptions.
*   For future analyses involving `CLUSTER` and `WP_` IDs, consider creating a definitive mapping file between these identifier types at the outset. This would streamline the process and reduce potential ambiguity or errors during data integration.
